In [1]:
#@Imports

#############################
# region: Imports and Setup #
#############################
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload
#############################
# endregion                 #
#############################

import datetime
from datetime import datetime as dt
from datetime import timedelta as td

from concordia.associative_memory import importance_function
from concordia.components import agent as components

from examples.custom_components.setup import model_setup, measurement_setup
from examples.tpb import utils
from examples.tpb import components as tpb_components
from examples.tpb import memory as tpb_memory
from examples.tpb import plan as tpb_plan
from examples.tpb import agent as tpb_agent

In [2]:
SETUP_TIME=dt(year=2024,month=10,day=1,hour=18)
TIME_STEP=td(minutes=5)

model, embedder = model_setup("llama3")
measurement, clock = measurement_setup(setup_time=SETUP_TIME,time_step=TIME_STEP)

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

In [3]:
config = tpb_agent.AgentConfig(
  name="Alice",
  gender="female",
  context="Alice loves to cook, but doesn't know what a Space Shuttle is.",
  goal="Alice wants to become a patissier.",
  traits="Determined, scatterbrained, picky",
  extras={
    'model': model,
    'embedder': embedder,  
    'importance': importance_model.importance,
    'clock_now': clock.now
    }
)

As a toddler, Alice was already determined to make her mark on the world. She would spend hours in her play kitchen, whipping up imaginary meals for her stuffed animals and demanding that they sit down to eat. Her parents would laugh at her antics, but it was clear that this little girl had a passion for cooking that would only grow stronger as she got older. Despite being scatterbrained, Alice's creativity shone through in her play, and she would often get lost in her own little world of make-believe.

As Alice grew into a young woman, her pickiness became more pronounced. She would spend hours perfecting the perfect recipe, only to declare that it was "almost" good enough. Her friends and family would chuckle at her dedication, but deep down, they knew that this was what drove Alice's creativity and innovation. When she wasn't cooking up a storm in her tiny kitchen, Alice could be found daydreaming about the perfect meal or scribbling down recipe ideas in her notebook. Despite being 

In [4]:
params = {
  "model": model,
  "config": config,
  "num_memories": 100,
  "clock_now": clock.now,
  "verbose": True,
  "num_behavs": 5,
  "num_people": 5,
  "threading": True
}

# Memory
memory = tpb_memory.EpisodicMemory(**params)

# Situation
situation = components.situation_perception.SituationPerception(
  "situation",
  model=model,
  memory=config.memory,
  agent_name=config.name,
  clock_now = clock.now,
  num_memories_to_retrieve=100,
  verbose=True
)

# TPB pipeline components
behav = tpb_components.Behaviour("behaviour", **params)
attitude = tpb_components.Attitude("attitude", components=[behav], **params)
people = tpb_components.People("people", components=[behav], **params)
motiv = tpb_components.Motivation("motivation", components=[people], **params)
norm = tpb_components.SubjectiveNorm("norm", components=[motiv], **params)
control = tpb_components.BehaviouralControl("control", components=[behav], **params)
thin_goal = tpb_components.ThinGoal("thin_goal", **params)
intention = tpb_components.BehaviouralIntention(
  "intention", 
  components=[attitude, norm, control, thin_goal], 
  **params
)
plan = tpb_plan.TPBPlan(
  "plan",
  goal = intention,
  components=[situation],
  **params
)

full_model = tpb_components.TPBModel(
  "full_model",
  components=[
    memory,
    behav,
    attitude,
    people,
    motiv,
    norm,
    control,
    thin_goal,
    situation,
    intention,
    plan
  ]
)

In [5]:
full_model.update()

[LOG] [Component] behaviour initialized.
[LOG] [Component] attitude initialized.
[LOG] [Component] people initialized.
[LOG] [Component] motivation initialized.
[LOG] [Component] control initialized.
[LOG] [Component] intention initialized.
[LOG] [Component] thin_goal initialized.
As Alice

 deliberated on whether to take the job offer, she considered the potential consequences of saying yes, including sacrificing time with loved ones and potentially impacting her relationships, versus saying no, which could lead to financial instability and regret for passing up a career opportunity.Based on the timeline provided, Alice is currently 38 years old (2001 + 23). Therefore, she is in a situation where she has likely gained significant experience and expertise in cooking, possibly even running her own kitchen or working as a patissier.Memories of Alice:
[23 Jul 1986 00:00:00] When Alice was 6 years old, she spent her birthday party trying to convince her parents to let her help with the cake-making. She insisted that she could do a better job than the bakery and even drew pictures of different designs on scraps of paper. Her parents humored her, but ultimately decided to stick with their own recipe.
[23 Jul 1989 00:00:00] When Alice was 9 years old, she discovered her mom's ol

In [6]:
full_model.update()

Based on Alice's personality and goals, here are five potential behaviors she could exhibit:

1. **Dive into cookbook research**: Given her love for cooking and desire to become a patissier, Alice might decide to spend some time researching different types of pastries and desserts, looking for inspiration and new techniques to try out.

2. **Experiment with new ingredients**: As someone who enjoys experimenting with recipes, Alice might choose to try out some new ingredients or flavor combinations in her cooking, potentially leading to the creation of a unique dessert or pastry recipe.

3. **Seek guidance from a professional**: Recognizing that she's still learning and wants to improve, Alice might decide to seek out guidance from an experienced patissier or chef, either through online tutorials, cookbooks, or even taking a class.

4. **Practice making different types of pastries**: With her goal of becoming a patissier in mind, Alice might focus on practicing making different types of

In [13]:
%autoreload 2
from examples.tpb.utils import pprint

full_model.state() >> pprint
intention.state() >> pprint

Here is Alice's revised plan for the rest of the day:

[18:00 - 19:00] Finish dinner preparations for tonight's family gathering, making sure to include some new recipes she's been experimenting with.

[19:00 - 20:00] Take a short break to relax and unwind after a long day in the kitchen. Maybe take a walk around the block or read a cookbook to get inspiration for future dishes.

[20:00 - 21:00] Start planning for tomorrow's patissier class, reviewing notes from previous classes and making a list of ingredients and equipment needed for the next project.

[21:00 - 22:00] Take some time to research and learn more about Space Shuttles, as she's still curious about what they are. Maybe watch a documentary or read an article online.

[22:00 - 23:00] Continue working on her patissier skills by practicing a new recipe or technique, making sure to take notes and refine her process.

[23:00 - 00:00] Wind down with some calming activities before bed, such as listening to music or doing some ligh